# Requerimiento 5

### Importamos las librerias necesarias

In [5]:
import pandas as pd
from typing import List, Tuple
from preprocessing import preprocess_text
from similarity_metrics import cosine_similarity_effective_lemmas, jaccard_similarity_effective_lemmas


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/esteban/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/esteban/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


Algunos datos de NLTK no se encontraron. Descargando...
Descargando punkt...
Descargando averaged_perceptron_tagger...
Descargando wordnet...
Descargando omw-1.4...
Descargando stopwords...
Descarga de datos NLTK completada.


### Cargamos los datos

In [6]:
df = pd.read_csv("../../resources/collection.csv")
df

,Type of reference,First Author,Secondary Author,Tertiary Author,Subsidiary Author,Abstract,Author Address,Accession Number,Author,Location in Archives,...,User definable 3.,User definable 4.,User definable 5.,URL,Volume number,Published Standard number,Primary Date,Access Date,End of Reference,Unnamed: 79
0,JOUR,NaN,NaN,NaN,NaN,The digital paradigm requires efficient method...,NaN,NaN,"Słyk, Jan",NaN,...,NaN,NaN,NaN,https://doi.org/10.1007/s00004-022-00603-1,24.0,NaN,NaN,NaN,,NaN
1,JOUR,NaN,NaN,NaN,NaN,Computational design and shape grammars hold a...,NaN,NaN,"El-Mahdy, Deena",NaN,...,NaN,NaN,NaN,https://doi.org/10.1007/s00004-022-00608-w,24.0,NaN,NaN,NaN,,NaN
2,JOUR,NaN,NaN,NaN,NaN,This paper proposes a hybrid design technique ...,NaN,NaN,"Lin, Jia-Yang",NaN,...,NaN,NaN,NaN,https://doi.org/10.1007/s00004-023-00657-9,25.0,NaN,NaN,NaN,,NaN
3,JOUR,NaN,NaN,NaN,NaN,This study presents a Translation of Form (ToF...,NaN,NaN,"Tan, Funda",NaN,...,NaN,NaN,NaN,https://doi.org/10.1007/s00004-024-00778-9,26.0,NaN,NaN,NaN,,NaN
4,JOUR,NaN,NaN,NaN,NaN,"In recent years, the gastronomy industry has i...",NaN,NaN,"Suarez, Juan Luis",NaN,...,NaN,NaN,NaN,https://doi.org/10.1007/s00146-021-01183-3,37.0,NaN,NaN,NaN,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,JOUR,NaN,NaN,NaN,NaN,One of the most unique and impressive feats of...,NaN,NaN,"Lieder, Falk",NaN,...,NaN,NaN,NaN,https://doi.org/10.3758/s13428-022-01789-5,55.0,NaN,NaN,NaN,,NaN
871,JOUR,NaN,NaN,NaN,NaN,Language models have been increasingly popular...,NaN,NaN,"Hou, Yubo",NaN,...,NaN,NaN,NaN,https://doi.org/10.3758/s13428-023-02313-z,56.0,NaN,NaN,NaN,,NaN
872,JOUR,NaN,NaN,NaN,NaN,"Generative AI, short for Generative Artificial...",NaN,NaN,"Markowitz, David M.",NaN,...,NaN,NaN,NaN,https://doi.org/10.3758/s13428-024-02344-0,56.0,NaN,NaN,NaN,,NaN
873,JOUR,NaN,NaN,NaN,NaN,"Over the last decade, continuous efforts have ...",NaN,NaN,"Lorincz, Beata",NaN,...,NaN,NaN,NaN,https://doi.org/10.7821/naer.2021.7.640,10.0,NaN,NaN,NaN,,NaN


### Creamos el primer abstract

In [7]:
def create_tupla(index: int, df) -> Tuple[str, List[str]]:
    """
    Crea una tupla con el índice, DOI y abstract.
    """
    abstract1_text = df.loc[index, 'Abstract']
    lemmas1 = preprocess_text(abstract1_text, language='english')
    return (df.loc[index, 'DOI'], lemmas1)
primer_tupla = create_tupla(0, df)


### Creamos una lista de tuplas con los demás abstracts

In [8]:
def create_list_tupla(index: int, df) -> List[Tuple[str, List[str]]]:
    """
    Crea una lista de tuplas con el índice, DOI y abstract.
    """
    list_tupla = []
    for i in range(index, len(df)):
        list_tupla.append(create_tupla(i, df))
    return list_tupla
list_tupla = create_list_tupla(1, df)

### Comparamos el primer abstract con los demás abstracts mediante dos metodos de similitud

1. **Metodos de similitud Coseno**

In [9]:
def compare_all_arts_with_cosine_similarity(art: Tuple[str, List[str]], list_simil: List[Tuple[str, List[str]]]
) -> List[Tuple[str, float]]:
    result = []
    for j in range(0, len(list_simil)):
        # Comparar los abstracts
        cos_sim_ij = cosine_similarity_effective_lemmas(art[1], list_simil[j][1])
        result.append((list_simil[j][0], cos_sim_ij))
    return result
result = compare_all_arts_with_cosine_similarity(primer_tupla, list_tupla)
df_results = pd.DataFrame(result, columns=["DOI", "Cosine Similarity"])
df_results


,DOI,Cosine Similarity
0,10.1007/s00004-022-00608-w,0.300574
1,10.1007/s00004-023-00657-9,0.095652
2,10.1007/s00004-024-00778-9,0.303852
3,10.1007/s00146-021-01183-3,0.054019
4,10.1007/s00146-021-01252-7,0.033562
...,...,...
869,10.3758/s13428-022-01789-5,0.106202
870,10.3758/s13428-023-02313-z,0.052541
871,10.3758/s13428-024-02344-0,0.105736
872,10.7821/naer.2021.7.640,0.127008


2. **Metodos de similitud Jaccard**

In [10]:
def compare_all_arts_with_jaccard_similarity(
        art: Tuple[str, List[str]], list_simil: List[Tuple[str, List[str]]]
) -> List[Tuple[str, float]]:
    result = []
    for j in range(0, len(list_simil)):
        # Comparar los abstracts
        jac_sim_ij = jaccard_similarity_effective_lemmas(art[1], list_simil[j][1])
        result.append((list_simil[j][0], jac_sim_ij))
    return result
result2 = compare_all_arts_with_jaccard_similarity(primer_tupla, list_tupla)
df_results2 = pd.DataFrame(result2, columns=["DOI", "Jaccard Similarity"])
df_results2


,DOI,Jaccard Similarity
0,10.1007/s00004-022-00608-w,0.129032
1,10.1007/s00004-023-00657-9,0.057554
2,10.1007/s00004-024-00778-9,0.129771
3,10.1007/s00146-021-01183-3,0.045752
4,10.1007/s00146-021-01252-7,0.025974
...,...,...
869,10.3758/s13428-022-01789-5,0.055944
870,10.3758/s13428-023-02313-z,0.036496
871,10.3758/s13428-024-02344-0,0.058824
872,10.7821/naer.2021.7.640,0.072993
